In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

/Users/aryamantepal/anaconda3/envs/ML-env/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/aryamantepal/anaconda3/envs/ML-env/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/aryamantepal/anaconda3/envs/ML-env/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <761A4B43-4CD1-322C-BB16-CEE783FE0A7C> /Users/aryamantepal/anaconda3/envs/ML-env/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


1. Basic autograd example 1      

In [2]:
# create tensors 

x = torch.tensor(1., requires_grad = True)
w = torch.tensor(2., requires_grad = True)
b = torch.tensor(3., requires_grad = True)

print(f"x: {x}\nw: {w}\nb: {b}")

x: 1.0
w: 2.0
b: 3.0


In [3]:
# building a computational graph
y  = w * x + b # y = 2 * x + 3

# compute gradients
y.backward()

In [4]:
# print out the gradients

print(f"Gradients:\nx: {x.grad}\nw: {w.grad}\nb: {b.grad}")

Gradients:
x: 2.0
w: 1.0
b: 1.0


2. Autograd Example 2

In [5]:
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# build a fully connected layer

linear = nn.Linear(3, 2)
print('w:', linear.weight)
print('b:', linear.bias)

w: Parameter containing:
tensor([[-0.2388, -0.5554,  0.0982],
        [ 0.0749,  0.4349,  0.4445]], requires_grad=True)
b: Parameter containing:
tensor([ 0.0014, -0.0958], requires_grad=True)


In [6]:
# build loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr = 0.01)

# forward pass
pred = linear(x)

# compute loss
loss = criterion(y, pred)
print('loss:', loss.item())

loss: 0.9903333783149719


In [7]:
# backward pass
loss.backward()

# print out the gradients
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

dL/dw:  tensor([[ 0.5118, -0.5877,  0.2504],
        [ 0.0279,  0.0048,  0.1055]])
dL/db:  tensor([-0.4457,  0.2287])


In [8]:
# 1-step gradient descent

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

optimizer.step()

In [9]:
pred = linear(x)
loss = criterion(y, pred)
print('loss after 1 step optimization:', loss.item())

loss after 1 step optimization: 0.9811269044876099


3. Loading data from numpy

In [11]:
# create a numpy array
x = np.array([[1, 2], [3, 4]])

# numpy array to a torch tensor
y = torch.from_numpy(x)

# torch tensor to a numpy array
z = y.numpy()

4. Input Pipeline

In [12]:
# cifar10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

100%|██████████| 170498071/170498071 [00:05<00:00, 32122141.14it/s]


Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [13]:
# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

torch.Size([3, 32, 32])
6


In [14]:
# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

In [15]:
# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

In [17]:
# Mini-batch images and labels.
images, labels = next(data_iter)

# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

5. Input pipeline for a custom dataset

In [19]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
# train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
#                                            batch_size=64, 
#                                            shuffle=True)

6. Using a pre-trained model

In [20]:
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 10)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())  

/Users/aryamantepal/anaconda3/envs/ML-env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/aryamantepal/anaconda3/envs/ML-env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([64, 10])
